# Interactive Pareto front for decision making in reservoir operation under uncertainty
In this section by means of an interactive example putting you into the role of a water system manager we will introduce the concept of decision making under uncertainty and different approaches to deal with forecast uncertainty.

<left><img src="Images/Dam.gif" width = "400px"><left>
    
## Decision making under uncertainty
Imagine that as a manager you receive every week both a hydroclimate and a demand forecast for the next 4 weeks and must define a pumped inflow policy for this period. Each forecast contains 10 members or possible future scenarios with the same probability of occurrence. Here we present 3 different approaches to address the problem of defining a policy under uncertainty.
    
Main sources of uncertainty for this example:
    
**1) Hydroclimate forecast**

<right> <img src="Images/Hydroclimatic_forecast3.gif" width = "400px"><right>

**2) Demand forecast**
    
<right> <img src="Images/Demand1.gif" width = "400px"><right>
    
## 1. Import libraries
**First of all, we need to import the necessary libraries to run the model:**
### 1.1 Mathematical functions: Numpy

In [1]:
import numpy as np

### 1.2 Interactive visualization: Bqplot and ipywidgets

In [2]:
from bqplot import pyplot as plt
from bqplot import *
from bqplot.traits import *
import ipywidgets as widgets

### 1.3 Optimization: Platypus (NSGA2)

In [3]:
from platypus import NSGAII, Problem, Real, Integer

## 2. Define of the reservoir system

<left> <img src="Images/system_representation_IO2.3.png" width = "600px"><left>

### 2.1 Inputs

#### 2.1.1 Variables:

$T$ = simulation time, in number of weeks

** Seasonal weather forecast **

$m$ = number of forecast members

$I$ = weekly natural inflows, in ML/week (Matrix of size [T, m])

$E$ = weekly evaporation volume, in ML/week (Matrix of size [T, m])

$d$ = weekly demand, in ML/week (Matrix of size [T, m])

In [4]:
T = 4 # weeks
members_num = 10
from clim_dem_forecast import forecast
I_for,temp_for,E_for,d_for,uncertain = forecast(T,members_num) # in ML/day

font_size = 14
x_sc_fe = LinearScale()
y_sc_fe = LinearScale(min = 0,max = 40)
x_ax_fe = Axis(label='week',scale=x_sc_fe,tick_values = np.array([1,2,3,4]),tick_style={'fill': 'black', 'font-size': 16})
y_ax_fe = Axis(label='ML/week', scale=y_sc_fe, orientation='vertical',tick_style={'fill': 'black', 'font-size': 16})

inflow_forecast = plt.plot(x=np.arange(1,T+1),y=I_for,colors=['blue'],stroke_width = 1,opacities = [0.5]*members_num,
                           display_legend=False,scales={'x': x_sc_fe, 'y': y_sc_fe})
fig_fe_a       = plt.Figure(marks = [inflow_forecast], title = 'Inflow forecast for the next 4 weeks', 
                    axes=[x_ax_fe, y_ax_fe],layout={'min_width': '1000px', 'max_height': '300px'},
                    scales={'x': x_sc_fe, 'y': y_sc_fe},legend_location = 'bottom-left')

demand_forecast = plt.plot(x=np.arange(1,T+1),y=d_for,colors=['gray'],stroke_width = 1,opacities = [0.5]*members_num,
                           display_legend=False,scales={'x': x_sc_fe, 'y': y_sc_fe})
fig_fe_b       = plt.Figure(marks = [demand_forecast], title = 'Demand forecast for the next 4 weeks', axes=[x_ax_fe, y_ax_fe],
                    layout={'min_width': '1000px', 'max_height': '300px'},
                    scales={'x': x_sc_fe, 'y': y_sc_fe},legend_location = 'bottom-left')

widgets.VBox([fig_fe_a,fig_fe_b])

A Jupyter Widget

** Decisions **

$u$ = pumped inflows, in ML/week (Vector of length T)


#### 2.1.2 Initial conditions:

$S_0$ = inital reservoir storage volume, in ML

In [5]:
S0 = 40 # in ML

#### 2.1.3 Constraints:

$S_{max}$ = max reservoir storage volume, in ML

In [6]:
Smax = 150 #  in ML

$S_{min}$ = min reservoir storage volume, in ML

In [7]:
Smin = 20 # in ML

$env_{min}$ = required environmental compensation flow, in ML/week

In [8]:
env_min = 2 # in ML/week

$c$ = pumping energy cost per ML, in £/ML

In [9]:
c = 50 # £/ML

### 2.2 Outputs

#### 2.2.1 Reservoir weekly storage

$S$ = reservoir storage, in ML (Vector of length T)

##### 2.2.2 Outflows
$u$ = reservoir controlled releases for water supply, in ML/week (Vector of length T)

$w$ = spillways, in ML/week (Vector of length T)

$env$ = environmental compensation outflow, in ML/week

## 3. Simulation + optimization
### 3.1 Reservoir system model

In [10]:
def syst_sim(T,I,E,d,S0,Smax,env_min):
    
    I = np.array(I)
    E = np.array(E)
    d = np.array(d)

    r = np.array(d)
    
    num_members = np.shape(I)[0]
    
    # Declare output variables

    S = np.array(np.zeros([np.shape(I)[0],np.shape(I)[1]+1]))

    w = np.array(np.zeros(np.shape(I)))

    env = np.array(np.zeros(np.shape(I)))+env_min
    
    ms = np.array(np.zeros(np.shape(I)[1]+1))+Smin
    
    pcost = np.array(np.zeros(np.shape(I)[0]))
    
    sdpen = np.array(np.zeros(np.shape(I)[0]))
    
    for m in range(num_members):
        
        S[m,0] = S0

        for t in range(T):

            # If at day t the inflow (I) is lower than the required environmental compensation (env_min), 
            # then environmental compensation (env) = inflow (I)  
            if env_min >= I[m,t] :
                env[m,t] = I[m,t]

            if env_min >= S[m,t] + I[m,t] - E[m,t]:
                env[m,t] = max(0,S[m,t] + I[m,t] - E[m,t])

            if d[m,t] >= S[m,t] + I[m,t] - E[m,t] - env[m,t]:
                r[m,t] = min(r[m,t],max(0,S[m,t] + I[m,t] - E[m,t] - env[m,t]))

            w[m,t] = max(0,S[m,t] + I[m,t] - r[m,t] - env[m,t] - E[m,t] - Smax)

            S[m,t+1] = S[m,t] + I[m,t] - r[m,t] - env[m,t]- E[m,t] - w[m,t]
              
    return S,env,w,r

### 3.2 Optimization

In [11]:
from platypus import NSGAII, Problem, Real, Integer

def auto_optim(vars):
    
    pinflow1 = vars[0]
    pinflow2 = 0
    pinflow3 = 0
    pinflow4 = 0
    
    u = np.array([pinflow1,pinflow2,pinflow3,pinflow4])
    S,env,w,r = syst_sim(T,I_for+u,E_for,d_for,S0,Smax,env_min)
    
    sdpen_mean = np.mean(np.sum(np.maximum(d_for-r,np.zeros(np.shape(d_for))),axis=1))
    pcost = np.sum(np.array(u)*c)
    
    return [sdpen_mean,pcost]

problem = Problem(1,2)
Real0 = Real(0, 40)

problem.types[:] = [Real0]
problem.function = auto_optim

population_size = 20
algorithm = NSGAII(problem,population_size)
algorithm.run(10000) # Number of iterations

results1_optim_relea = np.array([algorithm.result[i].objectives[0] for i in range(population_size)])
results2_optim_relea = np.array([algorithm.result[i].objectives[1] for i in range(population_size)])

solutions_optim_relea = [[algorithm.result[i].variables[0],0,0,0] for i in range(population_size)]

## 4. Results visualization + Decision making

** Pareto front **

<left> <img src="Images/ParetoFront.png" width = "600px"><left>

In [14]:
sdpen = np.zeros([members_num,population_size])
sdpen_mean = np.zeros(population_size)
sdpen_std = np.zeros(population_size)

for i in range(population_size):
    S_opt,env_opt,w_opt,r_opt = syst_sim(T,I_for+solutions_optim_relea[i],E_for,d_for,S0,Smax,env_min)
    sdpen[:,i] = np.sum(np.maximum(d_for-r_opt,np.zeros(np.shape(d_for))),axis = 1)
    sdpen_mean[i] = np.mean(sdpen[:,i])
    sdpen_std[i] = np.std(sdpen[:,i])
    r_opt

In [13]:
def update_operation(i):
    S,env,w,r    = syst_sim(T,I_for+solutions_optim_relea[i],E_for,d_for,S0,Smax,env_min)
    fig_wd.title = 'Supply deficit - Probability = {:.0f}'.format(np.max(np.count_nonzero(d_for-r,axis =0)))+' / '+str(members_num)
    fig_pi.title = 'Pumped inflow - Energy cost = £{:.0f}'.format(results2_optim_relea[i])
    return       S,solutions_optim_relea[i],r,results1_optim_relea[i],results2_optim_relea[i],i

def solution_selected(change):
    if pareto_front.selected == None:
        pareto_front.selected = [0]
    storage.y = update_operation(pareto_front.selected[0])[0]
    deficit.y = np.maximum(d_for-update_operation(pareto_front.selected[0])[2],np.zeros(np.shape(d_for)))
    pinflows.y = update_operation(pareto_front.selected[0])[1]
    pareto_front_ensemble.x = np.reshape([results2_optim_relea for i in range(0, members_num)],(members_num,population_size))[:,pareto_front.selected[0]]
    pareto_front_ensemble.y = sdpen[:,pareto_front.selected[0]]
    pareto_front_ensemble.unselected_style={'opacity': 0.1}
    pareto_front_ensemble.selected_style={'opacity': 0.1}
    pareto_front_ensemble.opacity = [0.1]*10
    
x_sc_pf = LinearScale()
y_sc_pf = LinearScale(min = 0,max = 50)

x_ax_pf = Axis(label='Pumping energy cost [£]', scale=x_sc_pf)
y_ax_pf = Axis(label='Supply deficit [ML]', scale=y_sc_pf, orientation='vertical')

pareto_front = plt.scatter(results2_optim_relea[:],results1_optim_relea[:],scales={'x': x_sc_pf, 'y': y_sc_pf},colors=['deepskyblue'], interactions={'hover':'tooltip','click': 'select'})
pareto_front.unselected_style={'opacity': 0.8}
pareto_front.selected_style={'fill': 'red', 'stroke': 'yellow', 'width': '1125px', 'height': '125px'}

if pareto_front.selected == []:
    pareto_front.selected = [0]
    
pareto_front_ensemble = plt.Scatter(x=np.reshape([results2_optim_relea for i in range(0, members_num)],(members_num,population_size))[:,pareto_front.selected[0]],
                                    y=sdpen[:,pareto_front.selected[0]],scales={'x': x_sc_pf, 'y': y_sc_pf},
                                    colors=['red'], interactions={'hover':'tooltip','click': 'select'})
pareto_front_ensemble.unselected_style={'opacity': 0.1}
pareto_front_ensemble.selected_style={'opacity': 0.1}
pareto_front_ensemble.opacity = [0.1]*10
fig_pf = plt.Figure(marks=[pareto_front,pareto_front_ensemble],title = 'Pareto front', axes=[x_ax_pf, y_ax_pf],layout={'width': '500px', 'height': '500px'}, 
                    animation_duration=500)

pareto_front.observe(solution_selected,'selected')    

S,env,w,r    = syst_sim(T,I_for+solutions_optim_relea[pareto_front.selected[0]],E_for,d_for,S0,Smax,env_min)

x_sc_pi    = OrdinalScale(min=1,max=T);y_sc_pi = LinearScale(min=0,max=40); x_ax_pi = Axis(label='week', scale=x_sc_pi);                              y_ax_pi = Axis(label='ML/week', scale=y_sc_pi, orientation='vertical')
x_sc_st    = LinearScale(min=0,max=T); y_sc_st = LinearScale(min=10,max=160);x_ax_st = Axis(label='week', scale=x_sc_st,tick_values=[0.5,1.5,2.5,3.5]);y_ax_st = Axis(label='ML', scale=y_sc_st, orientation='vertical')
x_sc_wd    = OrdinalScale(min=1,max=T);y_sc_wd = LinearScale(min=0,max=60); x_ax_wd = Axis(label='week', scale=x_sc_wd);                              y_ax_wd = Axis(label='ML/week', scale=y_sc_wd, orientation='vertical')

pinflows = plt.bar(np.arange(1,T+1),solutions_optim_relea[pareto_front.selected[0]],scales={'x': x_sc_pi, 'y': y_sc_pi},
                               colors=['orange'],opacities = [1],stroke = 'lightgray',
                               labels = ['pumped inflow'], display_legend = False)

fig_pi   = plt.Figure(marks = [pinflows],axes=[x_ax_pi, y_ax_pi],layout={'max_width': '480px', 'max_height': '250px'},
                    scales={'x': x_sc_pi, 'y': y_sc_pi}, animation_duration=1000,legend_location = 'bottom-right')

storage           = plt.plot(x=np.arange(0,T+1),y=S,scales={'x': x_sc_st, 'y': y_sc_st},
                              colors=['blue'], stroke_width = 0.1,
                              fill = 'bottom', fill_opacities = [0.1]*members_num)
max_storage       = plt.plot(x=np.arange(0,T+1),y=[Smax]*(T+1),colors=['red'],scales={'x': x_sc_st, 'y': y_sc_st})
max_storage_label = plt.label(text = ['Max storage'], x=[0],y=[Smax+10],colors=['red'])
fig_st            = plt.Figure(marks = [storage,max_storage,max_storage_label], title = 'Reservoir storage volume', 
                     axes=[x_ax_st, y_ax_st],layout={'width': '1000px', 'height': '350px'}, animation_duration=1000,scales={'x': x_sc_st, 'y': y_sc_st})

vertical_lines_wd = plt.vline([1,2,3],colors = ['black'])

deficit = plt.bar(np.arange(1,T+1),np.maximum(d_for-r,np.zeros(np.shape(r))),scales={'x': x_sc_wd, 'y': y_sc_wd},
                                colors=['red'],opacities = [0.7]*members_num*T,stroke = 'lightgray',
                                labels = ['release'], display_legend = False,type = 'grouped',base=0,align='center')
fig_wd = plt.Figure(marks = [deficit,vertical_lines_wd],axes=[x_ax_wd, y_ax_wd],layout={'max_width': '480px', 'max_height': '250px'},
                    scales={'x': x_sc_wd, 'y': y_sc_wd}, animation_duration=1000,legend_location = 'bottom-right')

storage.y  = update_operation(pareto_front.selected[0])[0]
deficit.y  = np.maximum(d_for-update_operation(pareto_front.selected[0])[2],np.zeros(np.shape(d_for)))
pinflows.y = update_operation(pareto_front.selected[0])[1]

storage.observe(solution_selected, ['x', 'y'])
deficit.observe(solution_selected, ['x', 'y'])
pinflows.observe(solution_selected, ['x', 'y'])

widgets.VBox([widgets.HBox([widgets.VBox([fig_pi,fig_wd]),fig_pf]),widgets.HBox([fig_st])])

A Jupyter Widget